In [5]:
import numpy as np
import pandas as pd
from scipy import sparse
import pickle


def recommend():
    # Import user games, steam game data and other user data
    myg = pd.read_excel("Data/mygames.xlsx")
    sg = pd.read_csv("Data/clean_sg.csv", index_col="appid")
    # Load from sparse file and column headers
    ug = sparse.load_npz("Data/User_Data_tr.npz")
    with open ("Data/g_list.txt", 'rb') as fp:
        mygam = pickle.load(fp)
    # Create dataframe with user games in
    mygames = pd.DataFrame(index=mygam)
    mygames["mg"] = [(int(col) in list(myg.appid))*1 for col in mygam]
    nmg = sum(mygames["mg"])
    # Convert dataframe to sparse matrix
    mg = sparse.coo_matrix(mygames.astype("int32").values).tocsr()
    # Dot product of matricies to count number of games both parties own
    prod = np.array(ug.dot(mg).sum(1))
    # Count number of games a user has
    ct = np.array(ug.sum(1))
    # Calc similarity score
    sim = prod**2 / ((nmg+1)*(ct+1))
    # Extract indicies of 10 best matches. 
    tt = pd.DataFrame(sim).nlargest(30, 0).sort_index()
    # Weight scores in matrix by similarity and then sum to get total score by game
    s = ug[tt.index].multiply(np.array([[i] for i in tt[0]])).sum(0)
    mygames["score"] = s.tolist()[0]
    # Join results with information frame
    mygames = mygames.join(sg)
    # Adjust score to reflect review positivity
    mygames["score"] = mygames["score"]*mygames["%pos"]
    return mygames.nlargest(1000, "score")


def filter_results(r,c):
    # Filter data based on user config settings
    if c["max_age"]:
        r = r.loc[r["required_age"]<=c["max_age"]]
    if c["language"]:
        r = r.loc[r["languages"].apply(lambda x: c["language"].lower() in str(x).lower())]
    if c["free_only"]:
        r = r.loc[r["price"]==0]
    if c["test_mode"] != True:
        r = r.loc[r["mg"]==0]
    return r.nlargest(c["per_page"]*c["page"], "score").iloc[-c["per_page"]:]


def format_output(r, c):
    # Set dataframe to desired format for display
    cols = ['name', '%pos', 'genre', 'price', 'discount']
    if c["test_mode"]:
        cols = ["mg", "score"]+cols  
    return r[cols]

    
def main(c):
    # Create recommendation scores
    recommendations = recommend()
    # Filter to match configs
    r = filter_results(recommendations, c)
    # Create output
    r = format_output(r, c)
    return r


# Configs
configs = {
    "max_age"   : False,
    "language"  : "English",
    "free_only" : False,
    "per_page"  : 10,
    "page"      : 1,
    "test_mode" : True
}

r = main(configs)
r

,mg,score,name,%pos,genre,price,discount
212680,1,251.171777,FTL: Faster Than Light,96.2,"Indie, Simulation, Strategy",2.49,75
220,0,244.626133,Half-Life 2,97.2,Action,1.99,80
72850,1,238.826224,The Elder Scrolls V: Skyrim,94.6,RPG,19.99,0
240,0,233.312266,Counter-Strike: Source,96.0,Action,1.99,80
620,1,230.252481,Portal 2,98.8,"Action, Adventure",1.99,80
105600,1,224.555326,Terraria,97.8,"Action, Adventure, Indie, RPG",4.99,50
340,0,224.118362,Half-Life 2: Lost Coast,86.2,Action,0.00,0
8930,0,216.902652,Sid Meier's Civilization® V,96.1,Strategy,29.99,0
320,0,216.613451,Half-Life 2: Deathmatch,89.6,Action,0.99,80
233450,1,213.739155,Prison Architect,91.5,"Indie, Simulation, Strategy",7.49,75
